# Importing the libraries

In [1]:
!pip install scikit-learn==0.19.0

     |████████████████████████████████| 12.4MB 5.5MB/s eta 0:00:01
ERROR: category-encoders 2.0.0 has requirement scikit-learn>=0.20.0, but you'll have scikit-learn 0.19.0 which is incompatible.
ERROR: autoai-libs 1.10.5 has requirement numpy>=1.16.4, but you'll have numpy 1.15.4 which is incompatible.
ERROR: autoai-libs 1.10.5 has requirement pandas>=0.24.2, but you'll have pandas 0.24.1 which is incompatible.
ERROR: autoai-libs 1.10.5 has requirement scikit-learn==0.20.3, but you'll have scikit-learn 0.19.0 which is incompatible.
  Found existing installation: scikit-learn 0.20.3
    Uninstalling scikit-learn-0.20.3:
      Successfully uninstalled scikit-learn-0.20.3


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib
import sklearn.metrics as metrics
import types
from botocore.client import Config
import ibm_boto3


print('numpy', np.__version__)
print('pandas', pd.__version__)
print('matplotlib', matplotlib.__version__)
print('seaborn', sns.__version__)

numpy 1.15.4
pandas 0.24.1
matplotlib 3.0.2
seaborn 0.9.0


# Importing the dataset

In [3]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_62d16fde9ee64a49b809102844695402 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XSCASHjDajDxaWQPvUeAapDDHRz6wRS_Hd2vgjN-IDMf',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_62d16fde9ee64a49b809102844695402.get_object(Bucket='predictingtheenergyoutputofwindtu-donotdelete-pr-4zv0a0xlcw57fc',Key='Dataset.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_excel(body)
dataset.head()

,Date/Time,Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction,maxtempC,windGustKmph,DewPointC,windChillC,humidity,pressure,LV ActivePower
0,2018-01-01 00:00:00,5.311336,416.328908,259.994904,10,11.125,2.208333,6.333333,69.916667,1017.625,380.047791
1,2018-01-01 00:10:00,5.672167,519.917511,268.641113,10,11.125,2.208333,6.333333,69.916667,1017.625,453.769196
2,2018-01-01 00:20:00,5.216037,390.900016,272.564789,10,11.125,2.208333,6.333333,69.916667,1017.625,306.376587
3,2018-01-01 00:30:00,5.659674,516.127569,271.258087,10,11.125,2.208333,6.333333,69.916667,1017.625,419.645905
4,2018-01-01 00:40:00,5.577941,491.702972,265.674286,10,11.125,2.208333,6.333333,69.916667,1017.625,380.650696


# Data Preprocessing

In [4]:
# Separating Date and Time
dataset['Date/Time'] = pd.to_datetime(dataset['Date/Time'],format='%d %m %Y %H:%M')
dataset['Hour'] = dataset['Date/Time'].dt.hour
dataset['Minute'] = dataset['Date/Time'].dt.minute
dataset['Day'] = dataset['Date/Time'].dt.day
dataset['Month'] = dataset['Date/Time'].dt.month
dataset['Year'] = dataset['Date/Time'].dt.year
dataset = dataset.reindex(columns = ['Date/Time', 'Minute', 'Hour', 'Day', 'Month', 'Year', 'Wind Speed (m/s)' ,'Theoretical_Power_Curve (KWh)',
                                     'Wind Direction', 'maxtempC', 'windGustKmph', 'DewPointC', 'windChillC', 'humidity', 'pressure',
                                     'LV ActivePower'])
dataset.head()

,Date/Time,Minute,Hour,Day,Month,Year,Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction,maxtempC,windGustKmph,DewPointC,windChillC,humidity,pressure,LV ActivePower
0,2018-01-01 00:00:00,0,0,1,1,2018,5.311336,416.328908,259.994904,10,11.125,2.208333,6.333333,69.916667,1017.625,380.047791
1,2018-01-01 00:10:00,10,0,1,1,2018,5.672167,519.917511,268.641113,10,11.125,2.208333,6.333333,69.916667,1017.625,453.769196
2,2018-01-01 00:20:00,20,0,1,1,2018,5.216037,390.900016,272.564789,10,11.125,2.208333,6.333333,69.916667,1017.625,306.376587
3,2018-01-01 00:30:00,30,0,1,1,2018,5.659674,516.127569,271.258087,10,11.125,2.208333,6.333333,69.916667,1017.625,419.645905
4,2018-01-01 00:40:00,40,0,1,1,2018,5.577941,491.702972,265.674286,10,11.125,2.208333,6.333333,69.916667,1017.625,380.650696


In [5]:
dataset.shape

(50530, 16)

In [6]:
# Splitting the dataset into X and Y
X = dataset.iloc[:, 1:15].values
y = dataset.iloc[:, -1].values
print('Shape of X =', X.shape)
print('Shape of y =', y.shape)

Shape of X = (50530, 14)
Shape of y = (50530,)


In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

# Fitting xgboost Regression to the dataset

In [8]:
# Refitting xgboost Regression to the dataset
import xgboost as xgb
xgbr = xgb.XGBRegressor(base_score=0.6, booster='gbtree', n_estimators=60, n_jobs= -1)
xgbr.fit(X, y)

XGBRegressor(base_score=0.6, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=60,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
# Applying k-Fold Cross Validation (Which has lower RMSE)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = xgbr, X = X, y = y, cv = 10)
print('Accuracies mean = ',accuracies.mean())
print('Accuracies standard deviation = ' ,accuracies.std())

# Predciting Power from Custom User Input value

In [10]:
inputs = {0:'Minute', 1:'Hour', 2:'Day', 3:'Month', 4:'Year', 5:'Wind Speed (m/s)', 6:'Theoretical_Power_Curve (KWh)',
          7:'Wind Direction', 8:'maxtempC', 9:'windGustKmph', 10:'DewPointC', 11:'windChillC', 12:'humidity',
          13:'pressure'}

In [11]:
# Taking user input
a = []
for i in range(len(inputs)):
    print(inputs[i])
    a.append(input())

Minute
50
Hour
23
Day
31
Month
12
Year
2018
Wind Speed (m/s)
5.31
Theoretical_Power_Curve (KWh)
416.32
Wind Direction
260
maxtempC
10
windGustKmph
11
DewPointC
3
windChillC
6
humidity
70
pressure
1017


In [12]:
# Preprocesing the input
a = np.array(a)
a = a.astype('float64')
a = a.reshape(1, -1)

In [13]:
# prediction of result
result = xgbr.predict(a)
print('LV ActivePower =',result)

LV ActivePower = [2775.9265]


# Deployment of Model

## For Feature Scaling

In [14]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-07-15 04:16:17,979 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2020-07-15 04:17:24,657 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [15]:
feature_scaling = {
  "apikey": "FjGRsj0RfO9D1oQ4iSF9Txh2M4Eudeak_RK-0AscWPnQ",
  "iam_apikey_description": "Auto-generated for key 8c3e33f6-af44-4a96-823f-b59d2098d13c",
  "iam_apikey_name": "Service credentials-2",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/5e7c88de3bb9498ba003e31b3b4331f0::serviceid:ServiceId-e84a71fc-7172-4d1f-8d56-95cc7789671c",
  "instance_id": "add6b7cd-7881-485c-b8bf-2a625e8a2335",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [16]:
feature = WatsonMachineLearningAPIClient(feature_scaling)

In [17]:
metadata = {
    feature.repository.ModelMetaNames.AUTHOR_NAME : "MaheshTheng",
    feature.repository.ModelMetaNames.AUTHOR_EMAIL : "mahesh.theng@gmail.com",
    feature.repository.ModelMetaNames.NAME : "feature_scaling"
}

In [18]:
stored_data = feature.repository.store_model(sc_X, meta_props = metadata)

In [19]:
guid = feature.repository.get_model_uid(stored_data)

In [20]:
guid

'ef323d76-229a-4dcf-8fd0-e390adbcdfb6'

In [21]:
deploy = feature.deployments.create(guid)



#######################################################################################

Synchronous deployment creation for uid: 'ef323d76-229a-4dcf-8fd0-e390adbcdfb6' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='65ab8dd6-b526-40d0-bf86-255c60ea6543'
------------------------------------------------------------------------------------------------




In [22]:
feature.deployments.list()

------------------------------------  -------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                 TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
65ab8dd6-b526-40d0-bf86-255c60ea6543  Artifact deployment  online  DEPLOY_SUCCESS  2020-07-15T04:16:29.955Z  scikit-learn-0.19  model
2590f8a2-9dfa-405c-b16d-d1a1c7a5f133  Artifact deployment  online  DEPLOY_SUCCESS  2020-07-14T05:17:02.532Z  scikit-learn-0.19  model
------------------------------------  -------------------  ------  --------------  ------------------------  -----------------  -------------


In [23]:
scoring_endpoint = feature.deployments.get_scoring_url(deploy)

In [24]:
scoring_endpoint

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/add6b7cd-7881-485c-b8bf-2a625e8a2335/deployments/65ab8dd6-b526-40d0-bf86-255c60ea6543/online'

## For Model Deployment

In [25]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [26]:
model = {
  "apikey": "G5zJMG7FVoEnfJ1c6Znz4MVfbXN2mkVlHc-dN_K2mmuh",
  "iam_apikey_description": "Auto-generated for key 9e5d0dc9-ff8d-4ad7-a8ff-0d6a16ace06e",
  "iam_apikey_name": "Service credentials-3",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/5e7c88de3bb9498ba003e31b3b4331f0::serviceid:ServiceId-9a364dec-bf99-4098-b074-1af23334fdec",
  "instance_id": "add6b7cd-7881-485c-b8bf-2a625e8a2335",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [27]:
model = WatsonMachineLearningAPIClient(model)

In [28]:
metadata = {
    model.repository.ModelMetaNames.AUTHOR_NAME : "Mahesh",
    model.repository.ModelMetaNames.AUTHOR_EMAIL : "mahesh.theng@gmail.com",
    model.repository.ModelMetaNames.NAME : "ML_Model"
}

In [29]:
stored_data = model.repository.store_model(xgbr, meta_props = metadata)

In [30]:
guid = model.repository.get_model_uid(stored_data)

In [31]:
guid

'ddf80817-3ee7-4038-9efc-4c83fab00875'

In [32]:
deploy = model.deployments.create(guid)



#######################################################################################

Synchronous deployment creation for uid: 'ddf80817-3ee7-4038-9efc-4c83fab00875' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='eb529457-ae7c-48ac-8bcb-60fe841045ff'
------------------------------------------------------------------------------------------------




In [33]:
model.deployments.list()

------------------------------------  -------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                 TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
eb529457-ae7c-48ac-8bcb-60fe841045ff  Artifact deployment  online  DEPLOY_SUCCESS  2020-07-15T04:17:36.713Z  scikit-learn-0.19  model
65ab8dd6-b526-40d0-bf86-255c60ea6543  Artifact deployment  online  DEPLOY_SUCCESS  2020-07-15T04:16:29.955Z  scikit-learn-0.19  model
2590f8a2-9dfa-405c-b16d-d1a1c7a5f133  Artifact deployment  online  DEPLOY_SUCCESS  2020-07-14T05:17:02.532Z  scikit-learn-0.19  model
------------------------------------  -------------------  ------  --------------  ------------------------  -----------------  -------------


In [23]:
scoring_endpoint = model.deployments.get_scoring_url(deploy)

In [24]:
scoring_endpoint

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/add6b7cd-7881-485c-b8bf-2a625e8a2335/deployments/2590f8a2-9dfa-405c-b16d-d1a1c7a5f133/online'